In [4]:
import torch
import torch.nn as nn

### Download dataset

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-10 11:17:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.58MB/s    in 0.2s    

2024-03-10 11:17:18 (6.58 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### Encode dataset

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# map from chars to ints and vice versa
stoi = {c:i for i, c in enumerate(chars)}
itos = {i:c for i, c in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join([itos[i] for i in l])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [12]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:300])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### Train / validation split

In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

len(train_data), len(val_data)

(1003854, 111540)

### Train on chunks of data from random locations in dataset

In [14]:
block_size = 8  # aka context length
train_data[:block_size+1]   # block_size number of examples

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context}, the target is: {target}')

When input is tensor([18]), the target is: 47
When input is tensor([18, 47]), the target is: 56
When input is tensor([18, 47, 56]), the target is: 57
When input is tensor([18, 47, 56, 57]), the target is: 58
When input is tensor([18, 47, 56, 57, 58]), the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [45]:
torch.manual_seed(1337)
batch_size = 4  # number of independent sequences we will process in parallel
block_size = 8  # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [22]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'When input is {context.tolist()}, target is: {target}')

When input is [24], target is: 43
When input is [24, 43], target is: 58
When input is [24, 43, 58], target is: 5
When input is [24, 43, 58, 5], target is: 57
When input is [24, 43, 58, 5, 57], target is: 1
When input is [24, 43, 58, 5, 57, 1], target is: 46
When input is [24, 43, 58, 5, 57, 1, 46], target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], target is: 39
When input is [44], target is: 53
When input is [44, 53], target is: 56
When input is [44, 53, 56], target is: 1
When input is [44, 53, 56, 1], target is: 58
When input is [44, 53, 56, 1, 58], target is: 46
When input is [44, 53, 56, 1, 58, 46], target is: 39
When input is [44, 53, 56, 1, 58, 46, 39], target is: 58
When input is [44, 53, 56, 1, 58, 46, 39, 58], target is: 1
When input is [52], target is: 58
When input is [52, 58], target is: 1
When input is [52, 58, 1], target is: 58
When input is [52, 58, 1, 58], target is: 46
When input is [52, 58, 1, 58, 46], target is: 39
When input is [52, 58, 1, 58, 46, 39], targ

# Bigram language model (simplest LLM)

In [40]:
import torch
from torch import nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are (B, T) tensors of integers
        logits = self.token_embedding_table(idx) # (B, T, C) batch(4) x time(8) x channel(65)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T).  
        # For each batch, generate next predicted token to get (B, T+1).  
        # Do this max_new_tokens times.
        for _ in range(max_new_tokens):
            logits, loss = self(idx)                            # get preds.  logits is (B, T, C)
            logits = logits[:, -1, :]                           # focus only on last time step (B, C)
            probs = F.softmax(logits, dim=-1)                   # apply softmax to get probabilities (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)             # (B, T+1)
            
        return idx

    


In [41]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

generated_chars = decode(
    m.generate(
        idx=torch.zeros((1, 1), dtype=torch.long), 
        max_new_tokens=100
    )[0].tolist()
)

generated_chars

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3"

# Train the model

In [43]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)   # typically 3e-4 is a good learning rate for more complex problems

In [52]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.462167739868164


In [55]:
generated_chars = decode(
    m.generate(
        idx=torch.zeros((1, 1), dtype=torch.long), 
        max_new_tokens=500
    )[0].tolist()
)

print(generated_chars)


PUCAr Set; O:

I?
THas ik bediverow's,
Shie ied my! s.
Bounelt d, be o t hyo t immy s; musane?
W: LOLe ithe un our: st qus beee t:
Br s?
Heise thay by, ch he m ppres d; s me hearis fy is cond f IONIFovecheam drenche OFithaves orelint plou hemerug
Wh mbil d atlachand herat I whotansiulleetes h
Tavens! myowoure onacelosount asime s d pare path t onn s;
Orel r nshaller cowin okinghe Bror
Find he prcimathegedooshow:
Cownounr's I hagr seratx. he t wanerear t by tit thodselegrt R:
Fo


r bunour ther w


In [8]:
nn.Embedding(8, 32)(torch.arange(8)).shape

torch.Size([8, 32])